In [ ]:
import os
import pandas as pd
import pymannkendall as mk
from tqdm import tqdm

# Define Constants
base_input_path = r"Z:\Thesis\Data\ML_Data"
output_path = r"Z:\Thesis\Data\ML_Stats"
os.makedirs(output_path, exist_ok=True)

# Get all parquet files in the base input path
all_files = [
    os.path.join(root, file)
    for root, _, files in os.walk(base_input_path)
    for file in files
    if file.endswith(".parquet")
]

# Process each file
for file_path in tqdm(all_files, desc="Processing Files"):
    try:
        # Load the data
        data = pd.read_parquet(file_path)
        tqdm.write(f"Successfully loaded: {file_path}")

        # Identify if the file is surface or pressure
        if "surface" in file_path:
            file_type = "surface"
            variables = [
                't2m_min', 't2m_max', 't2m_mean',
                'd2m_min', 'd2m_max', 'd2m_mean',
                'u10_min', 'u10_max', 'u10_mean',
                'v10_min', 'v10_max', 'v10_mean',
                'ssrd_min', 'ssrd_max', 'ssrd_mean',
                'strd_min', 'strd_max', 'strd_mean',
                'e_min', 'e_max', 'e_mean',
                'pev_min', 'pev_max', 'pev_mean',
                'sst_min', 'sst_max', 'sst_mean',
                'sp_min', 'sp_max', 'sp_mean',
                'msl_min', 'msl_max', 'msl_mean',
                'cape_min', 'cape_max', 'cape_mean'
            ]
        elif "pressure" in file_path:
            file_type = "pressure"
            variables = [
                'z_min', 'z_max', 'z_mean',
                'r_min', 'r_max', 'r_mean',
                't_min', 't_max', 't_mean',
                'u_min', 'u_max', 'u_mean',
                'v_min', 'v_max', 'v_mean',
                'w_min', 'w_max', 'w_mean',
                'vo_min', 'vo_max', 'vo_mean'
            ]
        else:
            tqdm.write(f"Skipping unknown file type: {file_path}")
            continue

        # For pressure data, process each level
        if file_type == "pressure":
            levels = data['level'].unique()
            for level in levels:
                level_data = data[data['level'] == level]

                # Perform analysis for each variable
                for variable in variables:
                    if variable not in level_data.columns:
                        continue

                    try:
                        mk_result = mk.original_test(level_data[variable])
                        sens_slope = mk.sens_slope(level_data[variable])

                        # Save results
                        results_file = os.path.join(output_path, f"{os.path.basename(file_path).replace('.parquet', '')}_level_{level}_stats.csv")
                        results_df = pd.DataFrame({
                            'Variable': [variable],
                            'Level': [level],
                            'MK Trend': [mk_result.trend],
                            'MK p-value': [mk_result.p],
                            'MK Tau': [mk_result.Tau],
                            'Sens Slope': [sens_slope.slope],
                            'Sens Intercept': [sens_slope.intercept]
                        })
                        if not os.path.exists(results_file):
                            results_df.to_csv(results_file, index=False)
                        else:
                            results_df.to_csv(results_file, mode='a', header=False, index=False)
                    except Exception as e:
                        tqdm.write(f"Error processing {variable} at level {level} in {file_path}: {e}")

        # For surface data, process variables directly
        elif file_type == "surface":
            for variable in variables:
                if variable not in data.columns:
                    continue

                try:
                    mk_result = mk.original_test(data[variable])
                    sens_slope = mk.sens_slope(data[variable])

                    # Save results
                    results_file = os.path.join(output_path, f"{os.path.basename(file_path).replace('.parquet', '')}_stats.csv")
                    results_df = pd.DataFrame({
                        'Variable': [variable],
                        'MK Trend': [mk_result.trend],
                        'MK p-value': [mk_result.p],
                        'MK Tau': [mk_result.Tau],
                        'Sens Slope': [sens_slope.slope],
                        'Sens Intercept': [sens_slope.intercept]
                    })
                    if not os.path.exists(results_file):
                        results_df.to_csv(results_file, index=False)
                    else:
                        results_df.to_csv(results_file, mode='a', header=False, index=False)
                except Exception as e:
                    tqdm.write(f"Error processing {variable} in {file_path}: {e}")

    except Exception as e:
        tqdm.write(f"Error loading file {file_path}: {e}")


Processing Files:   0%|                                     | 0/238 [00:00<?, ?it/s]

Successfully loaded: Z:\Thesis\Data\ML_Data\1980\Bahrain_1980_pressure_monthly_stats_merged.parquet


Processing Files:   0%|                             | 1/238 [00:02<11:24,  2.89s/it]

Successfully loaded: Z:\Thesis\Data\ML_Data\1980\Bahrain_1980_surface_monthly_stats_merged.parquet


Processing Files:   1%|▏                            | 2/238 [00:03<05:52,  1.50s/it]

...

Successfully loaded: Z:\Thesis\Data\ML_Data\1981\Kuwait_1981_pressure_monthly_stats_merged.parquet


Processing Files:   7%|█▍                  | 17/238 [11:42:06<72:54:57, 1187.77s/it]

Successfully loaded: Z:\Thesis\Data\ML_Data\1981\Kuwait_1981_surface_monthly_stats_merged.parquet


Processing Files:   8%|█▌                   | 18/238 [11:42:14<50:54:21, 833.01s/it]

Successfully loaded: Z:\Thesis\Data\ML_Data\1981\Oman_1981_pressure_monthly_stats_merged.parquet


In [ ]:
import os
import pandas as pd
import pymannkendall as mk
from tqdm import tqdm

# Define Constants
base_input_path = r"Z:\Thesis\Data\ML_Data"
output_path = r"Z:\Thesis\Data\ML_Stats"
os.makedirs(output_path, exist_ok=True)

# Get all parquet files in the base input path
all_files = [
    os.path.join(root, file)
    for root, _, files in os.walk(base_input_path)
    for file in files
    if file.endswith(".parquet")
]

# Chunk size for processing
chunk_size = 100000

# Process each file
for file_path in tqdm(all_files, desc="Processing Files"):
    try:
        # Load the data
        data = pd.read_parquet(file_path)
        tqdm.write(f"Successfully loaded: {file_path}")

        # Extract year and country from the file name
        file_name = os.path.basename(file_path)
        parts = file_name.split('_')
        year = parts[1]
        country = parts[0]

        # Create output subfolder for year and country
        year_country_output_path = os.path.join(output_path, year, country)
        os.makedirs(year_country_output_path, exist_ok=True)

        # Identify if the file is surface or pressure
        if "surface" in file_path:
            file_type = "surface"
            variables = [
                't2m_min', 't2m_max', 't2m_mean',
                'd2m_min', 'd2m_max', 'd2m_mean',
                'u10_min', 'u10_max', 'u10_mean',
                'v10_min', 'v10_max', 'v10_mean',
                'ssrd_min', 'ssrd_max', 'ssrd_mean',
                'strd_min', 'strd_max', 'strd_mean',
                'e_min', 'e_max', 'e_mean',
                'pev_min', 'pev_max', 'pev_mean',
                'sst_min', 'sst_max', 'sst_mean',
                'sp_min', 'sp_max', 'sp_mean',
                'msl_min', 'msl_max', 'msl_mean',
                'cape_min', 'cape_max', 'cape_mean',
                'DUSMASS_min', 'DUSMASS_max', 'DUSMASS_mean',
                'DUSMASS25_min', 'DUSMASS25_max', 'DUSMASS25_mean',
                'DUFLUXU_min', 'DUFLUXU_max', 'DUFLUXU_mean',
                'DUFLUXV_min', 'DUFLUXV_max', 'DUFLUXV_mean'
            ]
        elif "pressure" in file_path:
            file_type = "pressure"
            variables = [
                'z_min', 'z_max', 'z_mean',
                'r_min', 'r_max', 'r_mean',
                't_min', 't_max', 't_mean',
                'u_min', 'u_max', 'u_mean',
                'v_min', 'v_max', 'v_mean',
                'w_min', 'w_max', 'w_mean',
                'vo_min', 'vo_max', 'vo_mean',
                'DUCMASS_min', 'DUCMASS_max', 'DUCMASS_mean',
                'DUCMASS25_min', 'DUCMASS25_max', 'DUCMASS25_mean',
                'DUFLUXU_min', 'DUFLUXU_max', 'DUFLUXU_mean',
                'DUFLUXV_min', 'DUFLUXV_max', 'DUFLUXV_mean'
            ]
        else:
            tqdm.write(f"Skipping unknown file type: {file_path}")
            continue

        # For pressure data, process each level
        if file_type == "pressure":
            levels = data['level'].unique()
            for level in levels:
                level_data = data[data['level'] == level]

                # Process in chunks
                for start in range(0, len(level_data), chunk_size):
                    chunk = level_data.iloc[start:start + chunk_size]

                    # Perform analysis for each variable
                    for variable in variables:
                        if variable not in chunk.columns:
                            continue

                        try:
                            mk_result = mk.original_test(chunk[variable])
                            sens_slope = mk.sens_slope(chunk[variable])

                            # Save results
                            results_file = os.path.join(year_country_output_path, f"{file_name.replace('.parquet', '')}_level_{level}_stats.csv")
                            results_df = pd.DataFrame({
                                'Variable': [variable],
                                'Level': [level],
                                'MK Trend': [mk_result.trend],
                                'MK p-value': [mk_result.p],
                                'MK Tau': [mk_result.Tau],
                                'Sens Slope': [sens_slope.slope],
                                'Sens Intercept': [sens_slope.intercept]
                            })
                            if not os.path.exists(results_file):
                                results_df.to_csv(results_file, index=False)
                            else:
                                results_df.to_csv(results_file, mode='a', header=False, index=False)
                        except Exception as e:
                            tqdm.write(f"Error processing {variable} at level {level} in {file_path}: {e}")

        # For surface data, process variables directly
        elif file_type == "surface":
            # Process in chunks
            for start in range(0, len(data), chunk_size):
                chunk = data.iloc[start:start + chunk_size]

                for variable in variables:
                    if variable not in chunk.columns:
                        continue

                    try:
                        mk_result = mk.original_test(chunk[variable])
                        sens_slope = mk.sens_slope(chunk[variable])

                        # Save results
                        results_file = os.path.join(year_country_output_path, f"{file_name.replace('.parquet', '')}_stats.csv")
                        results_df = pd.DataFrame({
                            'Variable': [variable],
                            'MK Trend': [mk_result.trend],
                            'MK p-value': [mk_result.p],
                            'MK Tau': [mk_result.Tau],
                            'Sens Slope': [sens_slope.slope],
                            'Sens Intercept': [sens_slope.intercept]
                        })
                        if not os.path.exists(results_file):
                            results_df.to_csv(results_file, index=False)
                        else:
                            results_df.to_csv(results_file, mode='a', header=False, index=False)
                    except Exception as e:
                        tqdm.write(f"Error processing {variable} in {file_path}: {e}")

    except Exception as e:
        tqdm.write(f"Error loading file {file_path}: {e}")


Processing Files:   0%|                                     | 0/238 [00:00<?, ?it/s]

Successfully loaded: Z:\Thesis\Data\ML_Data\1980\Bahrain_1980_pressure_monthly_stats_merged.parquet


Processing Files:   0%|                             | 1/238 [00:04<17:20,  4.39s/it]

Successfully loaded: Z:\Thesis\Data\ML_Data\1980\Bahrain_1980_surface_monthly_stats_merged.parquet


Processing Files:   1%|▏                            | 2/238 [00:05<08:35,  2.19s/it]

Successfully loaded: Z:\Thesis\Data\ML_Data\1980\Kuwait_1980_pressure_monthly_stats_merged.parquet


In [2]:
import os
import pandas as pd
import pymannkendall as mk
from tqdm import tqdm

# Define Constants
base_input_path = r"Z:\Thesis\Data\ML_Data"
output_path = r"Z:\Thesis\Data\stat_analysis\era5_aod"
os.makedirs(output_path, exist_ok=True)

# Specify which countries and years to process
countries_to_process = [
    "Saudi_Arabia",
    #"Yemen",
    #"Oman",
    #"Qatar",
    #"Bahrain",
    "United_Arab_Emirates",
    #"Kuwait"
]
years_to_process = [str(year) for year in range(1980, 1981)]

# Chunk size defaults
DEFAULT_CHUNK_SIZE = 100_000
SAUDI_CHUNK_SIZE = 50_000  # use smaller chunk for Saudi Arabia

# Get all parquet files in the base input path
all_files = [
    os.path.join(root, file)
    for root, _, files in os.walk(base_input_path)
    for file in files
    if file.endswith(".parquet")
]

# Process each file
for file_path in tqdm(all_files, desc="Processing Files"):
    file_name = os.path.basename(file_path)
    
    # Extract year and country from the file name
    # (Adjust parsing logic if your file naming scheme changes)
    # Expecting something like "Saudi_Arabia_1980_surface_monthly_stats_merged.parquet"
    parts = file_name.split('_')
    
    if len(parts) < 2:
        tqdm.write(f"Skipping file with unexpected naming format: {file_name}")
        continue
    
    country = parts[0]
    year = parts[1]

    # Skip if not in our desired list
    if country not in countries_to_process:
        continue
    if year not in years_to_process:
        continue

    # Set chunk size based on country
    if country == "Saudi_Arabia":
        chunk_size = SAUDI_CHUNK_SIZE
    else:
        chunk_size = DEFAULT_CHUNK_SIZE
    
    # Attempt to load data
    try:
        data = pd.read_parquet(file_path)
        tqdm.write(f"Successfully loaded: {file_path}")
    except Exception as e:
        tqdm.write(f"Error loading file {file_path}: {e}")
        continue
    
    # Create output subfolder for year and country
    year_country_output_path = os.path.join(output_path, year, country)
    os.makedirs(year_country_output_path, exist_ok=True)

    # Identify if the file is surface or pressure
    if "surface" in file_path:
        file_type = "surface"
        variables = [
            't2m_min', 't2m_max', 't2m_mean',
            'd2m_min', 'd2m_max', 'd2m_mean',
            'u10_min', 'u10_max', 'u10_mean',
            'v10_min', 'v10_max', 'v10_mean',
            'ssrd_min', 'ssrd_max', 'ssrd_mean',
            'strd_min', 'strd_max', 'strd_mean',
            'e_min', 'e_max', 'e_mean',
            'pev_min', 'pev_max', 'pev_mean',
            'sst_min', 'sst_max', 'sst_mean',
            'sp_min', 'sp_max', 'sp_mean',
            'msl_min', 'msl_max', 'msl_mean',
            'cape_min', 'cape_max', 'cape_mean',
            'DUSMASS_min', 'DUSMASS_max', 'DUSMASS_mean',
            'DUSMASS25_min', 'DUSMASS25_max', 'DUSMASS25_mean',
            'DUFLUXU_min', 'DUFLUXU_max', 'DUFLUXU_mean',
            'DUFLUXV_min', 'DUFLUXV_max', 'DUFLUXV_mean'
        ]
    elif "pressure" in file_path:
        file_type = "pressure"
        variables = [
            'z_min', 'z_max', 'z_mean',
            'r_min', 'r_max', 'r_mean',
            't_min', 't_max', 't_mean',
            'u_min', 'u_max', 'u_mean',
            'v_min', 'v_max', 'v_mean',
            'w_min', 'w_max', 'w_mean',
            'vo_min', 'vo_max', 'vo_mean',
            'DUCMASS_min', 'DUCMASS_max', 'DUCMASS_mean',
            'DUCMASS25_min', 'DUCMASS25_max', 'DUCMASS25_mean',
            'DUFLUXU_min', 'DUFLUXU_max', 'DUFLUXU_mean',
            'DUFLUXV_min', 'DUFLUXV_max', 'DUFLUXV_mean'
        ]
    else:
        tqdm.write(f"Skipping unknown file type: {file_path}")
        continue

    # ====================
    # PROCESS PRESSURE DATA
    # ====================
    if file_type == "pressure":
        levels = data['level'].unique()
        for level in levels:
            level_data = data[data['level'] == level]

            # Process in chunks
            for start in range(0, len(level_data), chunk_size):
                chunk = level_data.iloc[start:start + chunk_size]

                # Perform analysis for each variable
                for variable in variables:
                    if variable not in chunk.columns:
                        continue
                    try:
                        mk_result = mk.original_test(chunk[variable])
                        sens_slope = mk.sens_slope(chunk[variable])

                        # Prepare result data
                        results_df = pd.DataFrame({
                            'Variable': [variable],
                            'Level': [level],
                            'MK Trend': [mk_result.trend],
                            'MK p-value': [mk_result.p],
                            'MK Tau': [mk_result.Tau],
                            'Sens Slope': [sens_slope.slope],
                            'Sens Intercept': [sens_slope.intercept]
                        })

                        # Save results
                        results_file = os.path.join(
                            year_country_output_path,
                            f"{file_name.replace('.parquet', '')}_level_{level}_stats.csv"
                        )
                        if not os.path.exists(results_file):
                            results_df.to_csv(results_file, index=False)
                        else:
                            results_df.to_csv(results_file, mode='a', header=False, index=False)

                    except Exception as e:
                        tqdm.write(f"Error processing {variable} at level {level} in {file_path}: {e}")

    # ====================
    # PROCESS SURFACE DATA
    # ====================
    elif file_type == "surface":
        # Process in chunks
        for start in range(0, len(data), chunk_size):
            chunk = data.iloc[start:start + chunk_size]

            for variable in variables:
                if variable not in chunk.columns:
                    continue
                try:
                    mk_result = mk.original_test(chunk[variable])
                    sens_slope = mk.sens_slope(chunk[variable])

                    # Prepare result data
                    results_df = pd.DataFrame({
                        'Variable': [variable],
                        'MK Trend': [mk_result.trend],
                        'MK p-value': [mk_result.p],
                        'MK Tau': [mk_result.Tau],
                        'Sens Slope': [sens_slope.slope],
                        'Sens Intercept': [sens_slope.intercept]
                    })

                    # Save results
                    results_file = os.path.join(
                        year_country_output_path,
                        f"{file_name.replace('.parquet', '')}_stats.csv"
                    )
                    if not os.path.exists(results_file):
                        results_df.to_csv(results_file, index=False)
                    else:
                        results_df.to_csv(results_file, mode='a', header=False, index=False)

                except Exception as e:
                    tqdm.write(f"Error processing {variable} in {file_path}: {e}")


Processing Files: 100%|███████████████████████| 238/238 [00:00<00:00, 120459.07it/s]


In [2]:
! pip install pymannkendall

In [1]:
#computes MK & SS for chunks does not produce sinlge output. 

import os
import pandas as pd
import pymannkendall as mk
from tqdm import tqdm

# Define Constants
base_input_path = r"Z:\Thesis\Data\ML_Data"
output_path = r"Z:\Thesis\Data\stat_analysis\era5_aod"
os.makedirs(output_path, exist_ok=True)

# Specify which countries and years to process
countries_to_process = ["Saudi_Arabia",
                        "United_Arab_Emirates",
                        "Yemen",
                        "Oman",
                        "Qatar",
                        "Bahrain",
                        "Kuwait"
                       ]
years_to_process = [
    "1980",
    "1981",
    "1982",
    "1983",
    "1984",
    "1985",
    "1986",
    "1987",
    "1988",
    "1993",
    "2016",
    "2017",
    "2018",
    "2019",
    "2020",
    "2021",
    "2022",
    "2023"
]

# Chunk size defaults
DEFAULT_CHUNK_SIZE = 100_000
SAUDI_CHUNK_SIZE = 50_000  # Use smaller chunk for Saudi Arabia

# Collect all Parquet files
all_files = [
    os.path.join(root, file)
    for root, _, files in os.walk(base_input_path)
    for file in files
    if file.endswith(".parquet")
]

print(f"Found {len(all_files)} files in {base_input_path}")

def parse_filename(file_name):
    """
    Parse the file name to extract country and year.
    We assume exactly one of the parts is a 4-digit year,
    and everything before that is the country name.
    Example:
      Saudi_Arabia_1980_surface_monthly_stats_merged.parquet
        => country='Saudi_Arabia', year='1980'
      United_Arab_Emirates_1981_pressure_monthly_stats_merged.parquet
        => country='United_Arab_Emirates', year='1981'
    """
    # Remove the ".parquet" extension
    base_name = file_name.replace(".parquet", "")
    parts = base_name.split("_")

    year_index = None
    year_str = None

    # Find the first part that looks like a 4-digit year
    for i, part in enumerate(parts):
        if part.isdigit() and 1900 <= int(part) <= 2100:
            year_index = i
            year_str = part
            break

    # If we didn't find a valid year, return None to signal invalid format
    if year_index is None:
        return None, None

    # Country is everything before the year
    country_name = "_".join(parts[:year_index])
    return country_name, year_str

# Process each file
for file_path in tqdm(all_files, desc="Processing Files"):
    file_name = os.path.basename(file_path)

    # Extract country and year
    country, year = parse_filename(file_name)
    if country is None or year is None:
        tqdm.write(f"Skipping file with unexpected naming format: {file_name}")
        continue

    tqdm.write(f"Extracted country: {country}, year: {year}")

    # Filter by specified countries
    if countries_to_process and (country not in countries_to_process):
        tqdm.write(f"Skipping file {file_path} for excluded country: {country}")
        continue

    # Filter by specified years
    if years_to_process and (year not in years_to_process):
        tqdm.write(f"Skipping file {file_path} for excluded year: {year}")
        continue

    # Set chunk size based on country
    chunk_size = SAUDI_CHUNK_SIZE if country == "Saudi_Arabia" else DEFAULT_CHUNK_SIZE

    # Attempt to load data
    try:
        data = pd.read_parquet(file_path)
        tqdm.write(f"Successfully loaded: {file_path}")
    except Exception as e:
        tqdm.write(f"Error loading file {file_path}: {e}")
        continue

    # Create output subfolder for year and country
    year_country_output_path = os.path.join(output_path, year, country)
    os.makedirs(year_country_output_path, exist_ok=True)

    # Identify if the file is surface or pressure
    if "surface" in file_name.lower():
        file_type = "surface"
        variables = [
            't2m_min', 't2m_max', 't2m_mean',
            'd2m_min', 'd2m_max', 'd2m_mean',
            'u10_min', 'u10_max', 'u10_mean',
            'v10_min', 'v10_max', 'v10_mean',
            'ssrd_min', 'ssrd_max', 'ssrd_mean',
            'strd_min', 'strd_max', 'strd_mean',
            'e_min', 'e_max', 'e_mean',
            'pev_min', 'pev_max', 'pev_mean',
            'sst_min', 'sst_max', 'sst_mean',
            'sp_min', 'sp_max', 'sp_mean',
            'msl_min', 'msl_max', 'msl_mean',
            'cape_min', 'cape_max', 'cape_mean',
            'DUSMASS_min', 'DUSMASS_max', 'DUSMASS_mean',
            'DUSMASS25_min', 'DUSMASS25_max', 'DUSMASS25_mean',
            'DUFLUXU_min', 'DUFLUXU_max', 'DUFLUXU_mean',
            'DUFLUXV_min', 'DUFLUXV_max', 'DUFLUXV_mean'
        ]

        # Process surface data in chunks
        for start in range(0, len(data), chunk_size):
            chunk = data.iloc[start : start + chunk_size]

            for variable in variables:
                if variable not in chunk.columns:
                    continue

                try:
                    mk_result = mk.original_test(chunk[variable])
                    sens_slope = mk.sens_slope(chunk[variable])

                    results_df = pd.DataFrame({
                        'Variable': [variable],
                        'MK Trend': [mk_result.trend],
                        'MK p-value': [mk_result.p],
                        'MK Tau': [mk_result.Tau],
                        'Sens Slope': [sens_slope.slope],
                        'Sens Intercept': [sens_slope.intercept]
                    })

                    results_file = os.path.join(
                        year_country_output_path,
                        f"{file_name.replace('.parquet', '')}_stats.csv"
                    )
                    if not os.path.exists(results_file):
                        results_df.to_csv(results_file, index=False)
                    else:
                        results_df.to_csv(results_file, mode='a', header=False, index=False)

                except Exception as e:
                    tqdm.write(f"Error processing {variable} in {file_path}: {e}")

    elif "pressure" in file_name.lower():
        file_type = "pressure"
        variables = [
            'z_min', 'z_max', 'z_mean',
            'r_min', 'r_max', 'r_mean',
            't_min', 't_max', 't_mean',
            'u_min', 'u_max', 'u_mean',
            'v_min', 'v_max', 'v_mean',
            'w_min', 'w_max', 'w_mean',
            'vo_min', 'vo_max', 'vo_mean',
            'DUCMASS_min', 'DUCMASS_max', 'DUCMASS_mean',
            'DUCMASS25_min', 'DUCMASS25_max', 'DUCMASS25_mean',
            'DUFLUXU_min', 'DUFLUXU_max', 'DUFLUXU_mean',
            'DUFLUXV_min', 'DUFLUXV_max', 'DUFLUXV_mean'
        ]

        # Get unique pressure levels
        levels = data['level'].unique()

        # Process pressure data by level
        for level in levels:
            level_filter = (data['level'] == level)

            # We still chunk to avoid loading huge data at once
            start_idx = 0
            data_len = level_filter.sum()  # number of rows at this level

            while start_idx < data_len:
                # Get indices for this chunk at the specified level
                # One approach is to use boolean indexing, but we must be careful with slicing.
                # We'll slice the *indices* that match 'level_filter'
                # Alternatively, we can subset level_data then chunk it normally.

                level_data = data[level_filter]  # all rows at this level
                chunk = level_data.iloc[start_idx : start_idx + chunk_size]
                start_idx += chunk_size

                for variable in variables:
                    if variable not in chunk.columns:
                        continue

                    try:
                        mk_result = mk.original_test(chunk[variable])
                        sens_slope = mk.sens_slope(chunk[variable])

                        results_df = pd.DataFrame({
                            'Variable': [variable],
                            'Level': [level],
                            'MK Trend': [mk_result.trend],
                            'MK p-value': [mk_result.p],
                            'MK Tau': [mk_result.Tau],
                            'Sens Slope': [sens_slope.slope],
                            'Sens Intercept': [sens_slope.intercept]
                        })

                        # Example filename:
                        # Bahrain_1980_pressure_monthly_stats_merged_level_10.0_stats.csv
                        results_file = os.path.join(
                            year_country_output_path,
                            f"{file_name.replace('.parquet', '')}_level_{level}_stats.csv"
                        )
                        if not os.path.exists(results_file):
                            results_df.to_csv(results_file, index=False)
                        else:
                            results_df.to_csv(results_file, mode='a', header=False, index=False)

                    except Exception as e:
                        tqdm.write(f"Error processing {variable} at level {level} in {file_path}: {e}")

    else:
        tqdm.write(f"Skipping unknown file type: {file_path}")


Found 238 files in Z:\Thesis\Data\ML_Data


Processing Files:   0%|                                     | 0/238 [00:00<?, ?it/s]

Extracted country: Bahrain, year: 1980
Successfully loaded: Z:\Thesis\Data\ML_Data\AP_training\1980\Bahrain_1980_pressure_monthly_stats_merged.parquet


Processing Files:   0%|                             | 1/238 [00:04<16:16,  4.12s/it]

Extracted country: Bahrain, year: 1980
Successfully loaded: Z:\Thesis\Data\ML_Data\AP_training\1980\Bahrain_1980_surface_monthly_stats_merged.parquet


...

Processing Files:  99%|██████████████████████▊| 236/238 [43:59:23<30:19, 909.68s/it]

Extracted country: Yemen, year: 2023
Successfully loaded: Z:\Thesis\Data\ML_Data\AP_training\2023\Yemen_2023_pressure_monthly_stats_merged.parquet


Processing Files: 100%|██████████████████████▉| 237/238 [44:05:41<12:30, 750.22s/it]

Extracted country: Yemen, year: 2023
Successfully loaded: Z:\Thesis\Data\ML_Data\AP_training\2023\Yemen_2023_surface_monthly_stats_merged.parquet


Processing Files: 100%|███████████████████████| 238/238 [44:06:33<00:00, 667.20s/it]
